In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_iris
from scipy.stats import chi2_contingency
import re
from datetime import datetime
from collections import Counter
from sklearn.preprocessing import MinMaxScaler


df = pd.read_pickle(r'/Users/maya/Documents/Information Systems/שנה ג׳/למידת מכונה/פרוייקט קורס/XY_train.pkl')
df.head()

,textID,text,sentiment,message_date,account_creation_date,previous_messages_dates,date_of_new_follower,date_of_new_follow,email,gender,email_verified,blue_tick,embedded_content,platform
0,b8f4e560fa,but yeah i like purple maybe thats why!! ;) ...,positive,2023-10-2 7:41:52,2015-4-13 0:1:16,"[2019-2-15 16:12:42, 2020-6-2 16:49:53, 2021-4...","[2016-1-13 7:12:26, 2016-5-17 13:40:38, 2016-1...","[2016-7-15 0:3:50, 2016-4-10 22:31:8, 2016-10-...",ClareRosindill3265@messenger.gov,F,NaN,False,mp4,facebook
1,f81a1511b2,_Henrie haha i WISH i coudl meet you.. you sho...,positive,2022-11-17 2:1:57,2013-7-1 18:58:16,"[2021-7-5 21:53:48, 2020-12-12 9:7:50, 2020-9-...","[2016-10-4 7:49:46, 2016-6-10 7:47:39, 2016-4-...","[2016-7-5 20:23:57, 2016-5-3 15:9:51, 2018-11-...",ErenaAntonchik231@messenger.gov,M,True,True,False,tiktok
2,3e9e3f0d69,"nah, you`re just altered forever Enjoy.",positive,2022-6-1 6:53:18,2013-8-8 2:21:12,"[2021-7-11 9:59:36, 2020-2-18 6:38:45, 2021-5-...","[2018-7-13 17:0:3, 2016-2-13 17:8:12, 2016-2-1...","[2016-7-9 22:15:10, 2018-4-13 20:37:55, 2016-1...",GodardCowlas6687@api.gov,F,True,True,False,tiktok
3,a068b95bd9,Can you ask Ryan why he stopped following me ...,negative,2022-1-4 11:38:57,2015-10-16 14:51:19,"[2021-8-16 4:55:32, 2021-6-1 10:13:37, 2021-12...",[2018-7-1 5:9:39],"[2018-10-18 7:23:20, 2018-4-10 12:17:6, 2016-6...",RalfStolberger6467@python.gov,M,False,False,mp4,tiktok
4,e6da2d1835,New issue of in the office...desperately want...,negative,2022-5-11 21:56:27,2013-12-15 19:8:40,"[2021-9-11 10:55:6, 2021-5-6 12:1:54, 2021-1-1...","[2018-11-2 11:16:12, 2018-10-18 16:8:34]","[2018-4-18 20:56:34, 2018-7-1 14:56:33, 2016-4...",MadelLeaton248@bbc.gov.il,M,False,False,None,None


## Pre-Processing Function

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.stem import SnowballStemmer

def preprocess_data(df):
    # Calculate missing values before deletion
    missing_values_before = df.isnull().sum()

    # Drop rows with missing values exceeding a threshold
    df = df.dropna(thresh=df.shape[1] - 2)

    missing_values_after = df.isnull().sum()
    missing_values_table = pd.DataFrame({
        'Before': missing_values_before,
        'After': missing_values_after,
        'Difference': missing_values_before - missing_values_after
    })
    print("Missing Values Before and After Deletion:")
    print(missing_values_table)

    # Fill missing values for 'email' with 'unknown'
    df['email'] = df['email'].fillna('unknown')

    # Fill missing values for 'embedded_content' and 'platform' based on their probability distributions
    embedded_content_prob = df['embedded_content'].value_counts(normalize=True)
    platform_prob = df['platform'].value_counts(normalize=True)

    def impute_missing_values(row, prob_dist):
        if pd.isnull(row):
            return np.random.choice(prob_dist.index, p=prob_dist.values)
        else:
            return row

    df['embedded_content'] = df['embedded_content'].apply(lambda x: impute_missing_values(x, embedded_content_prob))
    df['platform'] = df['platform'].apply(lambda x: impute_missing_values(x, platform_prob))

    # Fill missing values for 'email_verified' and 'blue_tick'
    df['email_verified'].fillna(df['blue_tick'], inplace=True)
    df['blue_tick'].fillna(df['email_verified'], inplace=True)

    # Fill missing values for 'gender' randomly
    df['gender'].replace('None', np.nan, inplace=True)
    gender_counts = df['gender'].value_counts()
    df['gender'].fillna(pd.Series(np.random.choice(gender_counts.index, size=len(df.index),
                                                   p=(gender_counts / gender_counts.sum()))), inplace=True)

    # Delete the remaining rows with missing values
    df = df.dropna()

    # Convert 'message_date' to categorical and create 'message_time_category' column
    df['message_date'] = pd.to_datetime(df['message_date'])
    df['hour'] = df['message_date'].dt.hour
    morning_interval = range(6, 12)  # 6:00 AM to 11:59 AM
    noon_interval = range(12, 18)    # 12:00 PM to 5:59 PM
    evening_interval = range(18, 24) # 6:00 PM to 11:59 PM

    def categorize_hour(hour):
        if hour in morning_interval:
            return 'Morning'
        elif hour in noon_interval:
            return 'Noon'
        elif hour in evening_interval:
            return 'Evening'
        else:
            return 'Night'

    df['message_time_category'] = df['hour'].apply(categorize_hour)
    df.drop(columns=['hour'], inplace=True)

    # Lower case
    df['text'] = df['text'].str.lower()

    # Stemming
    stemmer = SnowballStemmer('english')
    df['text'] = df['text'].apply(lambda x: ' '.join(stemmer.stem(word) for word in x.split()))

    # Remove punctuation
    df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

    # Remove numbers
    df['text'] = df['text'].apply(lambda x: re.sub(r'\d+', '', x))

    # Remove words with 1 letter
    df['text'] = df['text'].apply(lambda x: re.sub(r'\b\w{1}\b', '', x))

    # Remove top 0.05% of most common or not common words
    h_pct = 0.05
    l_pct = 0.05

    # Remove the top $h_pct of the most frequent words
    high_freq = pd.Series(' '.join(df['text']).split()).value_counts()[:int(pd.Series(' '.join(df['text']).split()).count() * h_pct / 100)]
    df['text'] = df['text'].apply(lambda x: ' '.join(x for x in x.split() if x not in high_freq))

    # Remove the top $l_pct of the least frequent words
    low_freq = pd.Series(' '.join(df['text']).split()).value_counts()[:-int(pd.Series(' '.join(df['text']).split()).count() * l_pct / 100):-1]
    df['text'] = df['text'].apply(lambda x: ' '.join(x for x in x.split() if x not in low_freq))

    # Remove double spaces
    df['text'] = df['text'].apply(lambda x: re.sub(r'\s+', ' ', x))

    return df

## Feature Extraction Function

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer

def extract_features(df):
    # 1. Create a new column based on the length of messages
    df['message_length'] = df['text'].apply(lambda x: len(x))

    # 2. Create the number of messages sent by the user
    df['num_messages_sent'] = df['previous_messages_dates'].apply(len)

    # 3. Create the number of followers and following
    df['follower_count'] = df['date_of_new_follower'].apply(lambda x: len(x))
    df['following_count'] = df['date_of_new_follow'].apply(lambda x: len(x))

    # 4. Append new columns created to the df
    new_columns_df = df[['follower_count', 'following_count']]

    # 5. N-GRAM
    X = df['clean_text']
    ngram_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=100)
    X_ngrams = ngram_vectorizer.fit_transform(X).toarray()
    df_output = pd.DataFrame(data=X_ngrams, columns=ngram_vectorizer.get_feature_names_out())

    # 6. Extract email domain endings
    def extract_email_domain_ending(email):
        if pd.isnull(email):
            return 'Missing'
        match = re.search(r'\.(\w+)$', email)
        if match:
            return match.group(1)
        else:
            return 'Unknown'
    df['email_domain_ending'] = df['email'].apply(extract_email_domain_ending)
    email_domain_ending_counts = df['email_domain_ending'].value_counts()

    # 7. Create seniority in years
    df['account_creation_date'] = pd.to_datetime(df['account_creation_date'])
    current_date = datetime.now()
    df['seniority'] = (current_date - df['account_creation_date']).dt.days / 365.25

    # 8. Create the average time difference between messages
    def calculate_average_time_difference(message_dates_array):
        message_dates = [datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S') for date_str in message_dates_array]
        time_diffs = []
        for i in range(1, len(message_dates)):
            time_diff = (message_dates[i] - message_dates[i - 1]).total_seconds()
            if time_diff < 0:
                time_diff = (message_dates[i - 1] - message_dates[i]).total_seconds()
            time_diffs.append(time_diff)
        if len(time_diffs) > 0:
            average_time_difference = sum(time_diffs) / len(time_diffs)
            average_time_difference = int(average_time_difference)
            return average_time_difference
        else:
            return None
    df['average_time_difference'] = df['previous_messages_dates'].apply(calculate_average_time_difference)

    return df, new_columns_df, df_output
# 
# # Assuming 'df' is your DataFrame
# df, new_columns_df, df_output = extract_features(df)


## Feature Representation Function

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def feature_representation(df, X_ngrams, ngram_vectorizer):
    # Normalize the values by dividing each column by its maximum value
    columns_to_normalize = ['message_length', 'num_messages_sent', 'follower_count', 'following_count', 'seniority']
    for column in columns_to_normalize:
        max_value = df[column].max()
        df[f'normalized_{column}'] = df[column] / max_value

    # One-hot coding
    email_domain_ending_onehot = pd.get_dummies(df['email_domain_ending'], prefix='email_ending')
    embedded_content_onehot = pd.get_dummies(df['embedded_content'], prefix='embedded_content')
    platform_onehot = pd.get_dummies(df['platform'], prefix='platform')
    message_time_category_onehot = pd.get_dummies(df['message_time_category'], prefix='message_time')

    df = pd.concat([df, email_domain_ending_onehot, embedded_content_onehot, platform_onehot, message_time_category_onehot], axis=1)
    df.drop(['email_domain_ending', 'embedded_content', 'platform', 'message_time_category'], axis=1, inplace=True)

    # Convert one hot coding to binary values
    bool_to_binary = {True: 1, False: 0}
    df['email_verified'] = df['email_verified'].map(bool_to_binary)
    df['blue_tick'] = df['blue_tick'].map(bool_to_binary)

    for column in df.columns:
        if df[column].dtype == bool:
            df[column] = df[column].astype(int) 

    gender_to_binary = {'F': 1, 'M': 0}
    df['gender'] = df['gender'].map(gender_to_binary)

    # Normalization for the n-gram features
    scaler = MinMaxScaler()
    X_ngrams_normalized = scaler.fit_transform(X_ngrams)
    df_normalized = pd.DataFrame(X_ngrams_normalized, columns=ngram_vectorizer.get_feature_names_out())
    df = pd.concat([df.reset_index(drop=True), df_normalized], axis=1)

    # Arrange data again 
    sentiment_mapping = {'positive': 1, 'negative': -1}
    df['sentiment'] = df['sentiment'].map(sentiment_mapping)
    df = df.drop(columns=['text', 'previous_messages_dates', 'message_date',
                          'email', 'date_of_new_follower', 'date_of_new_follow',
                          'account_creation_date', 'message_length', 'num_messages_sent',
                          'follower_count', 'following_count', 
                          'seniority', 'clean_text', 'average_time_difference'])

    return df


In [4]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


x = df.drop(columns=['sentiment', 'textID']) 
y = df['sentiment']

# Step 2: Initialize your model (e.g., Logistic Regression)
model = LogisticRegression()

# Step 3: Perform k-fold cross-validation using the processed dataset
k = 10 # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

for train_index, test_index in kf.split(df):
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]

# Visualize the dataset splitting for each fold
fold = 1
for train_index, test_index in kf.split(df):
    print(f"Fold {fold}:")
    print("Train indices:", train_index)
    print("Test indices:", test_index)
    print()
    fold += 1


Fold 1:
Train indices: [    0     1     2 ... 12268 12269 12271]
Test indices: [   14    19    31 ... 12229 12236 12270]

Fold 2:
Train indices: [    1     2     4 ... 12269 12270 12271]
Test indices: [    0     3     8 ... 12235 12254 12263]

Fold 3:
Train indices: [    0     1     2 ... 12269 12270 12271]
Test indices: [   10    23    29 ... 12239 12256 12265]

Fold 4:
Train indices: [    0     1     2 ... 12268 12269 12270]
Test indices: [   12    20    30 ... 12224 12241 12271]

Fold 5:
Train indices: [    0     1     2 ... 12269 12270 12271]
Test indices: [   26    27    51 ... 12215 12251 12260]

Fold 6:
Train indices: [    0     1     2 ... 12269 12270 12271]
Test indices: [   15    28    34 ... 12240 12261 12264]

Fold 7:
Train indices: [    0     1     3 ... 12269 12270 12271]
Test indices: [    2    18    43 ... 12259 12266 12268]

Fold 8:
Train indices: [    0     1     2 ... 12269 12270 12271]
Test indices: [    6     7    22 ... 12244 12249 12262]

Fold 9:
Train indices: [